In [1]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://poc-dask-cluster-1-scheduler:8786 Dashboard: http://poc-dask-cluster-1-scheduler:8787/status,Cluster Workers: 5 Cores: 40 Memory: 2.68 GB


In [2]:
import time
import random

def inc(x):
    time.sleep(random.random())
    return x + 1

def double(x):
    time.sleep(random.random())
    return 2 * x
    
def add(x, y):
    time.sleep(random.random())
    return x + y

In [3]:
inc(1)

2

In [4]:
future = client.submit(inc, 1)  # returns immediately with pending future
future

<Future: status: pending, key: inc-c22bd79e4d755edbfcfed7769cdd1e57>

In [5]:
future  # scheduler and client talk constantly

<Future: status: pending, key: inc-c22bd79e4d755edbfcfed7769cdd1e57>

In [6]:
future.result()

2

In [7]:
x = client.submit(inc, 1)
y = client.submit(double, 2)
z = client.submit(add, x, y)
z

<Future: status: pending, key: add-9d5502589292121c25a2bff600ca2cec>

In [8]:
z.result()

6

In [9]:
%%time
zs = []
for i in range(256):
    x = client.submit(inc, i)     # x = inc(i)
    y = client.submit(double, x)  # y = inc(x)
    z = client.submit(add, x, y)  # z = inc(y)
    zs.append(z)
    
total = client.submit(sum, zs)

CPU times: user 144 ms, sys: 24.5 ms, total: 168 ms
Wall time: 145 ms


In [10]:
L = zs
while len(L) > 1:
    new_L = []
    for i in range(0, len(L), 2):
        future = client.submit(add, L[i], L[i + 1])  # add neighbors
        new_L.append(future)
    L = new_L                                   # swap old list for new

In [11]:
del future, L, new_L, total  # clear out some old work

In [12]:
from dask.distributed import as_completed

zs = client.map(inc, zs)
seq = as_completed(zs)

while seq.count() > 2:  # at least two futures left
    a = next(seq)
    b = next(seq)
    new = client.submit(add, a, b)  # add them together
    seq.add(new)                    # add new future back into loop